## IBM設定

In [ ]:
channel = "ibm_quantum" # ibm_cloud, ibm_quantum
backend_name = "ibm_osaka"
token="" # あなたのトークンを入力してください

## ライブラリのインポート

In [ ]:
from qiskit import *
from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import numpy as np 
import matplotlib.pyplot as plt

import cv2

from common_logic import *
from common_quantum import *

%matplotlib inline

## プロパティ

In [ ]:
imageFileName = "test.jpg"
shot_count = 20000

## 実装

In [ ]:
# 画像読み込み
image = cv2.imread(f"{imageFileName}")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

width = len(gray_image[0])
height = len(gray_image)
w_qubits = width.bit_length()
h_qubits = height.bit_length()
n_qubits = w_qubits + h_qubits

In [ ]:
# 画像を量子状態にエンコード
probabilities = create_matrix(width,height)
for y in range(height):
    for x in range(width):
        probabilities[y][x] = np.sqrt(gray_image[y][x]/np.sum(gray_image))

address = gray_code_2d(width,height)

In [ ]:
# 量子回路を作成
state_init = convert_probability2statevector(probabilities,address)
qc = create_QuantumCircuit_by_Statevector(state_init)

In [ ]:
# 実機を実行
service = QiskitRuntimeService(channel=channel, token=token)
backend = service.backend(backend_name)
qc_transpiled = transpile(qc,backend)
sampler = Sampler(backend)
job = sampler.run([qc_transpiled],shots=shot_count)
result = job.result()
pub_result = result[0]
counts = pub_result.data.c.get_counts()

In [ ]:
# 量子状態を画像にデコード
image_quantumOutput = create_matrix(width,height)
for key,value in counts.items():
    x = gray_to_decimal(key[:w_qubits])
    y = gray_to_decimal(key[w_qubits:])
    val = clamp(float(np.sqrt(value*np.sum(gray_image)/(shot_count*255)))*255, 0, 255)
    if(x<width and y<height):
        image_quantumOutput[y][x] = val

cv2.imwrite('quantumOutput_remote.png',np.array(image_quantumOutput))

In [ ]:
# 画像を表示
plt.imshow(image_quantumOutput, cmap='gray')
plt.title('Gray Image')
plt.axis('off')
plt.show()